In [11]:
"""Variable Lifetime Analysis

Restriction:
- Only works for single function. Assume everything belongs to the same function (except for parameters that passing along).
- No global variable. We could do global variable at some other time, but no necessary for now.
"""

from functools import lru_cache
from pathlib import Path

import clang
import clang.cindex
from clang.cindex import CursorKind


def parse_file(path):
    idx = clang.cindex.Index.create()
    source_path = path
    home_path = Path.home()
    tu = idx.parse(
        path, args=[
            # On macos
            # '-I/usr/lib/clang/11/include',
            # '-std=c++17',
            # '-I/mnt/ssd2/junda/.pyenv/versions/3.10.6/include/python3.10',
            # '-I/home/junda/.pyenv/versions/3.10.6/lib/python3.10/site-packages/numpy/core/include',
            '-I/usr/include/python3.8',
            '-I/usr/lib/llvm-10/lib/clang/10.0.0/include',
            '-I/home/junda/.local/lib/python3.8/site-packages/numpy/core/include',
            '-std=c++17',
        ])
    for diag in tu.diagnostics:
        import warnings
        warnings.warn(str(diag))
    if tu.diagnostics:
        warnings.warn("Compiler error usually cause error to program analysis. If encountered downstream exception, consider look at function `parse_file` and alter the compiler flags.")
    return tu


def describe(node):
    return (node.kind.name, getattr(node.kind, "displayname", ""), node.spelling, node.extent)


def accessor(node, *pos):
    for p in pos:
        node = list(node.get_children())[p]
    return node


def traverse_children(node: 'clang.cindex.Cursor', depth=0):
    print('  ' * depth, *describe(node))
    for child in node.get_children():
        traverse_children(child, depth + 1)


def locate_function_decl(root, name: str):
    """
    Traverse the AST to find the function declaration with the spelling matching the `name`.
    :param root:
    :param name:
    :return:
    """

    def _traverse(node: 'clang.cindex.Cursor'):
        if node.kind == clang.cindex.CursorKind.FUNCTION_DECL and node.spelling == name:
            return node
        for child in node.get_children():
            result = _traverse(child)
            if result is not None:
                return result
        return None

    return _traverse(root)


def get_used_node_types(root):
    node_types = set()

    def get_all_used_node_types(node):
        node_types.add(node.kind)
        for child in node.get_children():
            get_all_used_node_types(child)

    get_all_used_node_types(root)
    return node_types


def traverse_children_with_life(root: 'clang.cindex.Cursor'):
    var_start = {}
    var_used = {}

    def _traverse_children_with_life(node: 'clang.cindex.Cursor', depth=0):

        if node.kind in NODE_DECL_TYPE:
            if node.spelling in var_start:
                print("Warning: variable {} is declared multiple times".format(node.spelling))
            var_start[node.spelling] = node
        if node.kind in NODE_REF_TYPE:
            var_used[node.spelling] = node

        # If node is a call expression, don't include the first element (function name).
        # Here we assume function name is always used.
        for i, child in enumerate(node.get_children()):
            if i == 0 and node.kind in [CursorKind.CALL_EXPR, CursorKind.FOR_STMT]:
                continue
            _traverse_children_with_life(child, depth + 1)
        return

    _traverse_children_with_life(root)
    return var_start, var_used


@lru_cache(maxsize=None)
def get_source_file(file_path):
    with open(file_path) as f:
        return f.read().splitlines(True)


def get_source_line(file_path, start, end):
    lines = get_source_file(file_path)
    return '\n'.join(lines[start.line - 1:end.line])


def describe_with_source(node):
    f = _main_func.extent.start.file.name
    return (f"{node.extent.start.line}:{node.extent.start.column}~{node.extent.end.line}:{node.extent.end.column}",
            get_source_line(f, node.extent.start, node.extent.end))


def find_path(root, target_node):
    # Traverse the root to find the path to the node.
    path = []

    def _traverse(node: 'clang.cindex.Cursor'):
        if node == target_node:
            path.append(node)
            return True
        for child in node.get_children():
            if _traverse(child):
                path.append(node)
                return True
        return False

    _trace = _traverse(root)
    return path[::-1]


def nearest_scope(path):
    # Find the nearest compound statement.
    result = path[0]
    for p in path:
        if p.kind in [CursorKind.FOR_STMT, CursorKind.WHILE_STMT, CursorKind.IF_STMT]:
            break
        if p.kind in [CursorKind.COMPOUND_STMT, CursorKind.FUNCTION_DECL]:
            result = p
            continue
        pass
    return result


def find_after_prefix(prefix_node, path):
    # Find the first node after the prefix.
    for i, p in enumerate(path):
        if p == prefix_node:
            return path[i:]
    else:
        return None


def get_node_enclose(x):
    return (x.line, x.column)


def is_node_enclosed(bigger_node, other):
    node = bigger_node
    return get_node_enclose(node.extent.start) <= get_node_enclose(other.extent.start) and get_node_enclose(
        node.extent.end) >= get_node_enclose(other.extent.end)


def find_next_sibling(scope_node, target_node):
    # Child may not be a direct child of the scope_node.
    # Try to find the node that encapsulate the child, then take the next sibling.
    children = list(scope_node.get_children())
    for i, child in enumerate(children):
        if i == len(children) - 1:
            return None
        if is_node_enclosed(child, target_node):
            return children[i + 1]
    return None


def find_enclosing_at_scope(scope_node, target_node):
    children = list(scope_node.get_children())
    for i, child in enumerate(children):
        if is_node_enclosed(child, target_node):
            return child
    return None


def suggest_delete_line(root, var_start, var_used):
    """Suggest deletion must happen at the enclose of the scope."""
    result = {}

    for var, node in var_start.items():
        if node.kind == CursorKind.PARM_DECL:
            continue
        if var not in var_used:
            # print(f"Unused variable {var}: ", *describe_with_source(node))
            result[var] = ('unused', node.extent.end.line)
            continue

        # Suggest the deletion line of the variable.
        used_node = var_used[var]
        used_path = find_path(root, used_node)
        used_scope = nearest_scope(used_path)
        for i, n in enumerate(used_path):
            if n == used_scope:
                a = used_path[i + 1]
                result[var] = ('last_used', a.extent.end.line)
                break

    return result


def describe_concise(node):
    return (
        node.kind.name, getattr(node.kind, "displayname", ""), node.spelling, node.extent.start.line,
        node.extent.end.line)


In [16]:
library_path = "/home/junda/.local/lib/python3.8/site-packages/clang/native/"
clang.cindex.Config.set_library_file(library_path + "libclang-16.so")

Exception: library file must be set before before using any other functionalities in libclang.

In [17]:



# args = parse_args()
# path = Path(args.source_path)
# target_function = args.target_function
path = Path("/home/junda/Scad/runtime/test/src/tpcds_1_cpp/reference_mem.cpp")
target_function = "main_"

tu = parse_file(path)
_main_func = locate_function_decl(tu.cursor, target_function)
if not _main_func:
    raise ValueError(f"Cannot find function {target_function}")

NODE_DECL_TYPE = [
    CursorKind.PARM_DECL,
    CursorKind.VAR_DECL,
]
NODE_REF_TYPE = [
    CursorKind.DECL_REF_EXPR,
]

NODE_SCOPE_TYPE = [
    CursorKind.FOR_STMT, CursorKind.WHILE_STMT, CursorKind.IF_STMT,
    CursorKind.COMPOUND_STMT, CursorKind.FUNCTION_DECL,
]

# [CursorKind.FOR_STMT, CursorKind.WHILE_STMT, CursorKind.IF_STMT]

# [CursorKind.COMPOUND_STMT, CursorKind.FUNCTION_DECL]

/tmp/ipykernel_996512/3548352135.py:34: UserWarning: /home/junda/Scad/runtime/test/src/tpcds_1_cpp/reference_mem.cpp:236:5: warning: implicit conversion of NULL constant to 'int' [-Wnull-conversion]
  warnings.warn(str(diag))
/tmp/ipykernel_996512/3548352135.py:36: UserWarning: Compiler error usually cause error to program analysis. If encountered downstream exception, consider look at function `parse_file` and alter the compiler flags.
  warnings.warn("Compiler error usually cause error to program analysis. If encountered downstream exception, consider look at function `parse_file` and alter the compiler flags.")


In [7]:
next_scope_id = 0
def traverse_children_with_life_scoped(root: 'clang.cindex.Cursor'):
    results = {}

    scope_def = [{}]
    scope_used = [{}]

    def __traverse(node: 'clang.cindex.Cursor', depth=0):
        global next_scope_id
        if node.kind in NODE_DECL_TYPE:
            if node.spelling:
                if node.spelling in scope_def[-1]:
                    print(
                        f"Warning: variable {node.spelling} is declared multiple times."
                    )
                scope_def[-1][node.spelling] = node
        

        if node.kind in NODE_REF_TYPE:
            # Node get used - find the original definition and mark it as used.
            for _scope_id in range(len(scope_def) - 1, -1, -1):
                if node.spelling in scope_def[_scope_id]:
                    scope_used[_scope_id][node.spelling] = node
                    break

        if node.kind == CursorKind.BINARY_OPERATOR:
            # Be careful and see if it is an assignment!
            opcode = parse_binary_op(node)
            print(f"Binary Operator: {node.spelling}, Opcode: {opcode}")
            pass
            

        # If node is a call expression, don't include the first element (function name).
        # Here we assume function name is always used.
        for i, child in enumerate(node.get_children()):
            if child.kind in [CursorKind.CALL_EXPR] and i == 0:
                continue
            if child.kind in NODE_SCOPE_TYPE:
                # enter a new scope
                scope_def.append({})
                scope_used.append({})

            __traverse(child, depth + 1)
            
            if child.kind in NODE_SCOPE_TYPE:
                # exit a scope
                a = scope_def.pop()
                b = scope_used.pop()
                if a or b:
                    results[next_scope_id] = (child, a, b)
                next_scope_id = next_scope_id + 1

        return

    __traverse(root)

    
    # Organize the result to get var_start, var_used.
    # But this time, `var_start` has to rename its variable with a line number as suffix ("@<line_number>")
    # ---> results # (scope_node_, scope_def_, scope_used_)
    var_start = {}
    var_used = {}
    for scope_id, (scope_node, scope_def, scope_used) in results.items():
        for var, node in scope_def.items():
            name = f"{var}@{node.extent.start.line}"
            var_start[name] = node
            if var in scope_used:
                var_used[name] = scope_used[var]

    return var_start, var_used, results

var_start, var_used, results__ = traverse_children_with_life_scoped(_main_func)

NameError: name '_main_func' is not defined

In [8]:
get_used_node_types(_main_func)

NameError: name '_main_func' is not defined

In [17]:
results__

{9: (<clang.cindex.Cursor at 0x7f0b3854cf40>,
  {'i': <clang.cindex.Cursor at 0x7f0b3853fdc0>},
  {'i': <clang.cindex.Cursor at 0x7f0b384da440>}),
 10: (<clang.cindex.Cursor at 0x7f0b384e30c0>,
  {'o_csv': <clang.cindex.Cursor at 0x7f0b3854cd40>,
   'o_dtype': <clang.cindex.Cursor at 0x7f0b3853f240>,
   'p_dict': <clang.cindex.Cursor at 0x7f0b3853f4c0>,
   'o_df': <clang.cindex.Cursor at 0x7f0b3853f2c0>,
   'o_tmp': <clang.cindex.Cursor at 0x7f0b3853f5c0>,
   'o_tmp2': <clang.cindex.Cursor at 0x7f0b3853f1c0>,
   'o_tmp3': <clang.cindex.Cursor at 0x7f0b3853fa40>,
   'o_proj': <clang.cindex.Cursor at 0x7f0b3853fb40>,
   'stride0': <clang.cindex.Cursor at 0x7f0b3853fc40>,
   'dim0': <clang.cindex.Cursor at 0x7f0b3853f8c0>,
   'data': <clang.cindex.Cursor at 0x7f0b3853f6c0>,
   'buffer': <clang.cindex.Cursor at 0x7f0b3853f7c0>},
  {'o_csv': <clang.cindex.Cursor at 0x7f0b3853f840>,
   'o_dtype': <clang.cindex.Cursor at 0x7f0b3853f940>,
   'p_dict': <clang.cindex.Cursor at 0x7f0b3853f740>,
 

In [18]:
var_start

{'i@290': <clang.cindex.Cursor at 0x7f0b3853fdc0>,
 'o_csv@267': <clang.cindex.Cursor at 0x7f0b3854cd40>,
 'o_dtype@270': <clang.cindex.Cursor at 0x7f0b3853f240>,
 'p_dict@272': <clang.cindex.Cursor at 0x7f0b3853f4c0>,
 'o_df@275': <clang.cindex.Cursor at 0x7f0b3853f2c0>,
 'o_tmp@279': <clang.cindex.Cursor at 0x7f0b3853f5c0>,
 'o_tmp2@280': <clang.cindex.Cursor at 0x7f0b3853f1c0>,
 'o_tmp3@281': <clang.cindex.Cursor at 0x7f0b3853fa40>,
 'o_proj@282': <clang.cindex.Cursor at 0x7f0b3853fb40>,
 'stride0@285': <clang.cindex.Cursor at 0x7f0b3853fc40>,
 'dim0@286': <clang.cindex.Cursor at 0x7f0b3853f8c0>,
 'data@287': <clang.cindex.Cursor at 0x7f0b3853f6c0>,
 'buffer@289': <clang.cindex.Cursor at 0x7f0b3853f7c0>,
 'o_csv@307': <clang.cindex.Cursor at 0x7f0b3854cac0>,
 'o_dtype@310': <clang.cindex.Cursor at 0x7f0b3854c9c0>,
 'p_dict@312': <clang.cindex.Cursor at 0x7f0b3854cdc0>,
 'j@364': <clang.cindex.Cursor at 0x7f0b384e2bc0>,
 'line@362': <clang.cindex.Cursor at 0x7f0b384e2940>,
 'i@361': 

In [19]:
var_used['o_df1@264']

In [20]:
var_end = suggest_delete_line(_main_func, var_start, var_used)

In [21]:
import pandas as pd
pd.DataFrame(var_end).T

,0,1
i@290,last_used,291
o_csv@267,last_used,273
o_dtype@270,last_used,273
p_dict@272,last_used,275
o_df@275,last_used,293
...,...,...
df5_len@419,last_used,450
df5_buf@420,last_used,447
o_df6@456,last_used,560
df6_buf@457,last_used,562
